In [1]:
import json
import os
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

from pathlib import Path

import math
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from sklearn.metrics import f1_score  

/home/jovyan/my-conda-envs/myenv/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/jovyan/my-conda-envs/myenv/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs
  warn(f"Failed to load image Python extension: {e}")


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Helper functions

In [3]:
def list_files(dir):                                                                                                  
    r = []                                                                                                            
    subdirs = [x[0] for x in os.walk(dir)]
    for subdir in subdirs:
        files = os.walk(subdir).__next__()[2]
        if (len(files) > 0):                                                                                          
            for file in files:
                r.append(os.path.join(subdir, file))                   
    return r

def custom_split(x):
    return [x[i:i+20].strip() for i in range(0, len(x), 20)]


### Paths

In [4]:
train_path='./CSV_Data/Training'
test_path='./CSV_Data/Performance Test'
validation_path='./CSV_Data/Validation'

### Preprocessing

In [5]:
class preprocess:
    def __init__(self,dir_path,start_index,end_index):
        self.x=None
        self.y = None
        self.vocab= dict()
        
        self.load(dir_path,start_index,end_index)
        self.tosplit()
        self.build_vocab()
        self.tokenize()
        print("#### Done ####")
        
    def len_vocab(self):
        return len(self.vocab)
    
    def load(self,dir_path,start_index,end_index):
        print('#### Loading files ####')
        files=list_files(dir_path)
        files.sort()
        files=files[start_index:end_index]
        dataframes=[]
        req_cols=["hex_values","class"]
        for file in files:
            print(file)
            df = pd.read_csv(file,sep='\t',usecols=req_cols)
            dataframes.append(df)
        data=pd.concat(dataframes,ignore_index=True)
        self.y=data['class']
        self.x=data['hex_values']
    
    def tosplit(self):
        for idx, value in self.x.iteritems():
            self.x[idx]=custom_split(value)
          
    def build_vocab(self):  
        print('#### Building vocab ####')     
        i=1
        for idx, value in self.x.iteritems():
            for element in value:
                if element in self.vocab:
                    pass
                else:
                    self.vocab[element]=i
                    i=i+1
            
    def tokenize(self):
        print('#### Tokenization ####')
        for idx, value in self.x.iteritems():
            for i in range(len(value)):
                try:
                    value[i]=self.vocab[value[i]]
                except:
                    value[i]=0
                
        

        
        

### Dataset

In [6]:
class classification_set(Dataset):
    def __init__(self,x,y):
        self.x=x
        self.y=y
        self.to_tensor()
    def __len__(self):
        return len(self.x)
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]
    def __setitem__(self,idx,value):
        self.x[idx]=value[0]
        self.y[idx]=value[1]
    def to_tensor(self):
        for i in range(len(self.x)):
            temp=list(self[i])
            temp[0]=torch.Tensor(temp[0]).int()
            temp[1]=float(temp[1])
            temp=tuple(temp)
            self[i]=temp
        

In [7]:
%%time
train_data=preprocess(train_path,21,29)

#### Loading files ####
./CSV_Data/Training/basic-V_7_8_P1-16.csv
./CSV_Data/Training/basic-V_7_8_P1-24.csv
./CSV_Data/Training/basic-V_7_8_P1-32.csv
./CSV_Data/Training/basic-V_7_8_P1-64.csv
./CSV_Data/Training/basic-V_7_9_P1-16.csv
./CSV_Data/Training/basic-V_7_9_P1-24.csv
./CSV_Data/Training/basic-V_7_9_P1-32.csv
./CSV_Data/Training/basic-V_7_9_P1-64.csv
#### Building vocab ####
#### Tokenization ####
#### Done ####
CPU times: user 31.8 s, sys: 2.21 s, total: 34.1 s
Wall time: 46.1 s


In [8]:
%%time
test_data=preprocess(test_path,0,9)

#### Loading files ####
./CSV_Data/Performance Test/V_7_1_P1-16.csv
./CSV_Data/Performance Test/V_7_1_P1-24.csv
./CSV_Data/Performance Test/V_7_1_P1-32.csv
./CSV_Data/Performance Test/V_7_8_P1-16.csv
./CSV_Data/Performance Test/V_7_8_P1-24.csv
./CSV_Data/Performance Test/V_7_8_P1-32.csv
./CSV_Data/Performance Test/V_7_9_P1-16.csv
./CSV_Data/Performance Test/V_7_9_P1-24.csv
./CSV_Data/Performance Test/V_7_9_P1-32.csv
#### Building vocab ####
#### Tokenization ####
#### Done ####
CPU times: user 5.21 s, sys: 125 ms, total: 5.34 s
Wall time: 7.2 s


In [9]:
# train_data.vocab.update(test_data.vocab)
# test_data.vocab=train_data.vocab
# test_data.tokenize()

In [10]:
# %%time
# validation_data=preprocess(validation_path)

In [9]:
train_set= classification_set(train_data.x,train_data.y)
test_set= classification_set(test_data.x,test_data.y)
# validation_set= classification_set(validation_data.x,validation_data.y)

In [12]:
# validation_dataloader = DataLoader(validation_set, batch_size)

In [10]:
len(test_data.vocab)

2929679

In [11]:
len(train_data.vocab)

14986772

### Playground

In [12]:
class classifier(nn.ModuleList):
    def __init__(self):
        super(classifier, self).__init__()
        self.num_words = 14986772
        self.embedding_size = 128
        self.seq_len = 200
        self.hidden_dim=64
        
        self.dropout=nn.Dropout(0.5)
        self.embeddings=nn.Embedding(self.num_words+1, self.embedding_size)
        self.flatten=nn.Flatten()
        
        self.kernel_1=4
        self.kernel_2=6
        
        self.stride = 2
        
        self.conv1 =nn.Conv1d(self.seq_len, self.hidden_dim, self.kernel_1, self.stride)
        self.conv2 =nn.Conv1d(self.hidden_dim, self.hidden_dim, self.kernel_2, self.stride)
        
        self.fc1 = nn.Linear(256, self.hidden_dim)
        self.fc2 = nn.Linear(self.hidden_dim, 1)
        
        self.pool_1 = nn.MaxPool1d(self.kernel_1, self.stride)
        self.pool_2 = nn.MaxPool1d(self.kernel_2, self.stride)
        
    def forward(self,x):
        

        x=self.embeddings(x)

        out=self.conv1(x)
        out=torch.relu(out)
        out=self.pool_1(out)
        out=self.dropout(out)

        out=self.conv2(out)
        out=torch.relu(out)
        out=self.pool_2(out)
        out=self.dropout(out)
        out=self.flatten(out)

        out=self.fc1(out)
        out=torch.relu(out)
        out=self.fc2(out)
        out=torch.sigmoid(out)

        return out.squeeze()
        
        
        
        
        
        
        

### Model

In [36]:
# import n_gram_cnn
# model= n_gram_cnn.model

In [13]:
model=classifier()

In [27]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [26]:
del model

In [15]:
print("allocated",torch.cuda.memory_allocated())
print("cached",torch.cuda.memory_reserved())

allocated 7673850368
cached 7675576320


In [14]:
model.to(device)

classifier(
  (dropout): Dropout(p=0.5, inplace=False)
  (embeddings): Embedding(14986773, 128)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (conv1): Conv1d(200, 64, kernel_size=(4,), stride=(2,))
  (conv2): Conv1d(64, 64, kernel_size=(6,), stride=(2,))
  (fc1): Linear(in_features=256, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
  (pool_1): MaxPool1d(kernel_size=4, stride=2, padding=0, dilation=1, ceil_mode=False)
  (pool_2): MaxPool1d(kernel_size=6, stride=2, padding=0, dilation=1, ceil_mode=False)
)

### Training / Evaluation


In [19]:
def train(model, train_set, test_set):
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    batch_size = 100
    train_dataloader = DataLoader(train_set, batch_size)
    test_dataloader = DataLoader(test_set, batch_size)
    
    for epoch in range(10):
        model.train()
        predictions1 =[]
        for x_batch, y_batch in train_dataloader:
            y_batch = y_batch.type(torch.FloatTensor)
            x_batch= x_batch.to(device)
            y_batch = y_batch.to(device)
            # x_batch= x_batch
            # y_batch = y_batch
            y_pred = model(x_batch)
            
            #remove squeeze in case batch size > 1
            y_batch=torch.squeeze(y_batch)

            
            loss = F.binary_cross_entropy(y_pred, y_batch)
            optimizer.zero_grad()
            loss.backward()
            # predictions.append(y_pred.cpu().detach().numpy())

            predictions1.extend(list(y_pred.cpu().detach().numpy()))
        predictions1=np.round_(predictions1)
        predictions = [int(a) for a in predictions1]


        test_predictions = evaluation(model, test_dataloader)
        train_accuary = calculate_accuray(train_set.y.to_list(), predictions1)
        test_accuracy = calculate_accuray(test_set.y.to_list(), test_predictions)
        # train_f1_score= calculate_f1_score(train_set.y.to_list(), predictions1)
        # test_f1_score= calculate_f1_score(test_set.y.to_list(), test_predictions)
        print("Epoch: %d, loss: %.5f, Train accuracy: %.5f,  Test accuracy: %.5f" % (epoch+1, loss.item(), train_accuary, test_accuracy))
        

def evaluation(model, test_dataloader):        
    model.eval()
    predictions2 = []
    # with torch.no_grad():
    for x_batch, y_batch in test_dataloader:
        
#         x_batch= x_batch
#         y_batch = y_batch        
        x_batch= x_batch.to(device)
        y_batch = y_batch.to(device)
        y_pred = model(x_batch)
        # predictions.append(y_pred.cpu().detach().numpy())
        predictions2.extend(list(y_pred.cpu().detach().numpy()))
    predictions2=np.round_(predictions2)
    predictions=[int(a) for a in predictions2]
    return predictions

def calculate_accuray(grand_truth, predictions):
    true_positives = 0
    true_negatives = 0
    for true, pred in zip(grand_truth, predictions):
        if (pred == 1 ) and (true == 1):
            true_positives += 1
        elif (pred == 0) and (true == 0):
            true_negatives += 1
        else:
            pass
    return (true_positives+true_negatives) / len(grand_truth)

def calculate_f1_score(grand_truth, predictions):
    return f1_score(grand_truth,predictions)


In [20]:
%%time
train(model,train_set,test_set)

RuntimeError: CUDA out of memory. Tried to allocate 7.15 GiB (GPU 0; 10.76 GiB total capacity; 7.16 GiB already allocated; 2.53 GiB free; 7.17 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF